In [1]:
import pandas as pd
import os

In [2]:
ros_data = []

In [3]:
# PLEASE CHANGE THE PATHS TO FOLDERS OF "logged_data" IN THIS VARIABLE

dirs = ["/home/sejego/TalTech/latency_article/results/ROS2/logged_data/publishers/",
        "/home/sejego/TalTech/latency_article/results/ROS2/logged_data/subscriptions/",
        "/home/sejego/TalTech/latency_article/results/ROS/logged_data/publishers/",
        "/home/sejego/TalTech/latency_article/results/ROS/logged_data/subscriptions/"]

In [4]:
for d in dirs:
    df = []
    for csv in os.scandir(d):
        df.append(pd.read_csv(csv))
    ros_data.append(df)

In [5]:
ros_data[1]

[            topic  received(#)  mean[ns]  max_latency  min_latency
 0  /input_current        58563    347716     18525159        47574
 1  /input_voltage        58563    356488     18489164        57766,
                     topic  received(#)  mean[ns]  max_latency  min_latency
 0  /electrical_torque_ref         3557    352572     12211065        71618
 1             /actual_rpm         3565    364128     11237504        91393,
                      topic  received(#)  mean[ns]  max_latency  min_latency
 0  /shaft_angular_velocity            0       NaN          NaN          NaN,
                            topic  received(#)  mean[ns]  max_latency  \
 0  /motor_power/electrical_power           11  350634.0     496630.0   
 1                    /efficiency          595  563932.0  110558995.0   
 2        /shaft_angular_velocity            0       NaN          NaN   
 
    min_latency  
 0     282568.0  
 1      99552.0  
 2          NaN  ,
 Empty DataFrame
 Columns: [topic, received(

In [6]:
ros2_pubs = pd.concat(ros_data[0], ignore_index=True)
ros2_subs = pd.concat(ros_data[1], ignore_index=True)
ros1_pubs = pd.concat(ros_data[2], ignore_index=True)
ros1_subs = pd.concat(ros_data[3], ignore_index=True)

In [7]:
ros1_pubs['topic'] = ros1_pubs['topic'].replace({"/electrical_torque": "/electrical_torque_ref"})
ros2_pubs['topic'] = ros2_pubs['topic'].replace({"/electrical_torque": "/electrical_torque_ref"})

In [8]:
ros1_pubs

,topic,sent(#)
0,/motor_power/electrical_power,11
1,/motor_power/reactive_power,3569
2,/efficiency,593
3,/actual_rpm,3556
4,/electrical_torque_ref,3568
5,/mechanical_torque,3568
6,/input_current,58536
7,/input_voltage,58536


In [9]:
ros1_subs

,topic,received(#),mean[ns],max_latency,min_latency
0,/input_current,58199,8.795962e+06,2.160330e+07,180411.0
1,/input_voltage,58427,8.837295e+06,2.200412e+07,227955.0
2,/electrical_torque_ref,3532,2.115738e+09,4.293061e+09,2982630.0
3,/actual_rpm,3532,2.114561e+09,4.293068e+09,3024210.0
4,/shaft_angular_velocity,0,NaN,NaN,NaN
5,/motor_power/electrical_power,11,2.500060e+09,4.131624e+09,558433100.0
6,/efficiency,588,1.582304e+07,1.703865e+07,174330.0
7,/shaft_angular_velocity,0,NaN,NaN,NaN


In [10]:
ros2_subs

,topic,received(#),mean[ns],max_latency,min_latency
0,/input_current,58563,347716.0,18525159.0,47574.0
1,/input_voltage,58563,356488.0,18489164.0,57766.0
2,/electrical_torque_ref,3557,352572.0,12211065.0,71618.0
3,/actual_rpm,3565,364128.0,11237504.0,91393.0
4,/shaft_angular_velocity,0,NaN,NaN,NaN
5,/motor_power/electrical_power,11,350634.0,496630.0,282568.0
6,/efficiency,595,563932.0,110558995.0,99552.0
7,/shaft_angular_velocity,0,NaN,NaN,NaN


In [11]:
ros2_pubs

,topic,sent(#)
0,/motor_power/electrical_power,11
1,/motor_power/reactive_power,3566
2,/efficiency,595
3,/actual_rpm,3567
4,/electrical_torque_ref,3565
5,/mechanical_torque,3565
6,/input_current,58566
7,/input_voltage,58566


In [12]:
ros1_full = pd.merge(ros1_subs,ros1_pubs, on = 'topic')
ros2_full = pd.merge(ros2_subs, ros2_pubs, on = 'topic')

In [13]:
ros1_full["mean[ns]"] = ros1_full["mean[ns]"] / 1000.0
ros2_full["mean[ns]"] = ros2_full["mean[ns]"] / 1000.0
ros2_full["max_latency"] = ros2_full["max_latency"] / 1000.0
ros1_full["max_latency"] = ros1_full["max_latency"] / 1000.0
ros2_full["min_latency"] = ros2_full["min_latency"] / 1000.0
ros1_full["min_latency"] = ros1_full["min_latency"] / 1000.0

In [14]:
ros1_full = ros1_full.rename(columns={"max_latency" : "max_latency[us]","min_latency": "min_latency[us]","mean[ns]" : "mean[us]"})
ros2_full = ros2_full.rename(columns={"max_latency" : "max_latency[us]","min_latency": "min_latency[us]","mean[ns]" : "mean[us]"})

In [15]:
ros2_full["lost(#)"] = ros2_full["sent(#)"] - ros2_full["received(#)"]
ros1_full["lost(#)"] = ros1_full["sent(#)"] - ros1_full["received(#)"]

In [16]:
ros1_full

,topic,received(#),mean[us],max_latency[us],min_latency[us],sent(#),lost(#)
0,/input_current,58199,8795.962,21603.305,180.411,58536,337
1,/input_voltage,58427,8837.295,22004.119,227.955,58536,109
2,/electrical_torque_ref,3532,2115737.754,4293061.017,2982.630,3568,36
3,/actual_rpm,3532,2114560.582,4293067.972,3024.210,3556,24
4,/motor_power/electrical_power,11,2500060.114,4131623.999,558433.100,11,0
5,/efficiency,588,15823.045,17038.650,174.330,593,5


In [17]:
ros2_full

,topic,received(#),mean[us],max_latency[us],min_latency[us],sent(#),lost(#)
0,/input_current,58563,347.716,18525.159,47.574,58566,3
1,/input_voltage,58563,356.488,18489.164,57.766,58566,3
2,/electrical_torque_ref,3557,352.572,12211.065,71.618,3565,8
3,/actual_rpm,3565,364.128,11237.504,91.393,3567,2
4,/motor_power/electrical_power,11,350.634,496.630,282.568,11,0
5,/efficiency,595,563.932,110558.995,99.552,595,0


In [18]:
message_sizes= [24,24,16,16,28,16]
ros2_full["message_size[b]"] = pd.DataFrame(message_sizes)
ros1_full["message_size[b]"] = pd.DataFrame(message_sizes)

In [19]:
ros2_full

,topic,received(#),mean[us],max_latency[us],min_latency[us],sent(#),lost(#),message_size[b]
0,/input_current,58563,347.716,18525.159,47.574,58566,3,24
1,/input_voltage,58563,356.488,18489.164,57.766,58566,3,24
2,/electrical_torque_ref,3557,352.572,12211.065,71.618,3565,8,16
3,/actual_rpm,3565,364.128,11237.504,91.393,3567,2,16
4,/motor_power/electrical_power,11,350.634,496.630,282.568,11,0,28
5,/efficiency,595,563.932,110558.995,99.552,595,0,16


In [20]:
# From iRobot
recv = [59849,59878,3595,3595,12,600]
means = [78,50,92,93,139,102]
maxs = [902,971,750,4912,233,796]
mins = [9,10,20,21,71,16]
losts =  [0,0,0,0,0,0,0]
msgs = [12,12,4,4,16,4]
freq = ['1000','1000','60','60','0.2','60']
irobot_full = ros2_full.copy()
irobot_full["received(#)"] = pd.DataFrame(recv)
irobot_full["mean[us]"] = pd.DataFrame(means)
irobot_full["max_latency[us]"] = pd.DataFrame(maxs)
irobot_full["min_latency[us]"] = pd.DataFrame(mins)
irobot_full["lost(#)"] = pd.DataFrame(losts)
irobot_full["sent(#)"] = irobot_full["received(#)"] + irobot_full["lost(#)"]
irobot_full["message_size[b]"] = pd.DataFrame(msgs)  + 16 # performance_test/header is 16 bytes
irobot_full["frequency[hz]"] = pd.DataFrame(freq)
ros2_full["frequency[hz]"] = irobot_full["frequency[hz]"]
ros1_full["frequency[hz]"] = irobot_full["frequency[hz]"]

In [21]:
irobot_full = irobot_full[["topic", "received(#)","sent(#)","lost(#)", "mean[us]","max_latency[us]","min_latency[us]","message_size[b]","frequency[hz]"]]
ros2_full = ros2_full[["topic", "received(#)","sent(#)","lost(#)", "mean[us]","max_latency[us]","min_latency[us]","message_size[b]","frequency[hz]"]]
ros1_full = ros1_full[["topic", "received(#)","sent(#)","lost(#)", "mean[us]","max_latency[us]","min_latency[us]","message_size[b]","frequency[hz]"]]


In [22]:
ros2_full = ros2_full.round(1)
ros1_full = ros1_full.round(1)

In [23]:
total = pd.DataFrame()

In [24]:
total = pd.DataFrame()
total["evaluations"] = pd.DataFrame(["iRobot's ROS2 framework","ROS2","ROS1"])
total["average_mean[us]"] = pd.DataFrame([irobot_full["mean[us]"].mean(),ros2_full["mean[us]"].mean(),ros1_full["mean[us]"].mean()])
total["lost(#)"] = pd.DataFrame([irobot_full["lost(#)"].sum(),ros2_full["lost(#)"].sum(),ros1_full["lost(#)"].sum()])
total["average_mean[us]"] = total["average_mean[us]"].round(1)

In [25]:
irobot_full = irobot_full.drop(columns="sent(#)")
ros2_full = ros2_full.drop(columns="sent(#)")
ros1_full = ros1_full.drop(columns="sent(#)")

In [26]:
irobot_full

,topic,received(#),lost(#),mean[us],max_latency[us],min_latency[us],message_size[b],frequency[hz]
0,/input_current,59849,0,78,902,9,28,1000
1,/input_voltage,59878,0,50,971,10,28,1000
2,/electrical_torque_ref,3595,0,92,750,20,20,60
3,/actual_rpm,3595,0,93,4912,21,20,60
4,/motor_power/electrical_power,12,0,139,233,71,32,0.2
5,/efficiency,600,0,102,796,16,20,60


In [27]:
ros2_full

,topic,received(#),lost(#),mean[us],max_latency[us],min_latency[us],message_size[b],frequency[hz]
0,/input_current,58563,3,347.7,18525.2,47.6,24,1000
1,/input_voltage,58563,3,356.5,18489.2,57.8,24,1000
2,/electrical_torque_ref,3557,8,352.6,12211.1,71.6,16,60
3,/actual_rpm,3565,2,364.1,11237.5,91.4,16,60
4,/motor_power/electrical_power,11,0,350.6,496.6,282.6,28,0.2
5,/efficiency,595,0,563.9,110559.0,99.6,16,60


In [28]:
ros1_full

,topic,received(#),lost(#),mean[us],max_latency[us],min_latency[us],message_size[b],frequency[hz]
0,/input_current,58199,337,8796.0,21603.3,180.4,24,1000
1,/input_voltage,58427,109,8837.3,22004.1,228.0,24,1000
2,/electrical_torque_ref,3532,36,2115737.8,4293061.0,2982.6,16,60
3,/actual_rpm,3532,24,2114560.6,4293068.0,3024.2,16,60
4,/motor_power/electrical_power,11,0,2500060.1,4131624.0,558433.1,28,0.2
5,/efficiency,588,5,15823.0,17038.6,174.3,16,60


In [40]:
print(total.to_latex())

\begin{tabular}{llrr}
\toprule
{} &              evaluations &  average\_mean[us] &  lost(\#) \\
\midrule
0 &  iRobot's ROS2 framework &              92.3 &        0 \\
1 &                     ROS2 &             389.2 &       16 \\
2 &                     ROS1 &         1127302.5 &      511 \\
\bottomrule
\end{tabular}



In [35]:
print(irobot_full.to_latex())

\begin{tabular}{llrrrrrrl}
\toprule
{} &                          topic &  received(\#) &  lost(\#) &  mean[us] &  max\_latency[us] &  min\_latency[us] &  message\_size[b] & frequency[hz] \\
\midrule
0 &                 /input\_current &        59849 &        0 &        78 &              902 &                9 &               28 &          1000 \\
1 &                 /input\_voltage &        59878 &        0 &        50 &              971 &               10 &               28 &          1000 \\
2 &         /electrical\_torque\_ref &         3595 &        0 &        92 &              750 &               20 &               20 &            60 \\
3 &                    /actual\_rpm &         3595 &        0 &        93 &             4912 &               21 &               20 &            60 \\
4 &  /motor\_power/electrical\_power &           12 &        0 &       139 &              233 &               71 &               32 &           0.2 \\
5 &                    /efficiency &          60